In [1]:
import cell_movie_maker as cmm
import chaste_simulation_database_connector as csdc
import matplotlib.pyplot as plt
import numpy as np
import pathlib


from IPython.display import display
import IPython.display


cmm.Config.simulations_folder = pathlib.Path("chaste_output")
cmm.Config.output_folder = pathlib.Path("analysis_output")

experiment = cmm.Experiment(pathlib.Path("chaste_output", "TCellABM"))
simulation = experiment.simulations[0]
sample_timepoint = simulation.timepoints[50]


# Analysis Framework Overview

The dataclass structure `Experiment`, `Simulation` and `SimulationTimepoint` help organise and retrieve CHASTE data.  
We can manually perform analysis on a `Simulation` or `SimulationTimepoint` by reading its underlying dataframe or numpy arrays, and when developing analysis methods this is often helpful.  
The analysis framework class structure is designed to facilitate applying this analysis to many simulations.  
This helps organise analysis, allows us to easily define how analysis can be parallelized, and makes it easier to store and retrieve analysis in a database.  

Whilst the general framework is highly flexible, the implemented analysis methods are specific to TCellABM (some methods may not work if a model is too different).  


## Preprocessors
`Preprocessor` is a class which can be used to perform quick preliminary analysis on a simulation.  
The idea behind `Preprocessor` is that it is extremely quick to run on a simulation and returns a single table (csv) for each simulation where each row is a timestep and each column is an analysis result at that timestep.  
This is only used for computing cellcounts in each frame. It is separate from the rest of the analysis framework mostly for legacy reasons, but could also be considered a very quick initial analysis of a parameter sweep, enough to plot cell population trajectories.  


## Analysers
`Analysers` make up the bulk of the analysis framework and the key method is `analyse` which takes as input some combination of `Simulation` and `SimulationTimepoint` and returns some analysis result. Note that the analysis result does not have to be a single number (often it is a `dict` or `Series`).  
`Analysers` may be `SimulationAnalyser`s, which analyse an entire `Simulation` (Often this means returning a `DataFrame` where each row is a timestep and each column is some analysis result), or `TimepointAnalyser`s, which analyse single `SimulationTimepoints`.  


# How to use Preprocessor
Note: This is a bit clunky to allow flexibility with different models

In [2]:
preprocessor = cmm.preprocessor.Preprocessor()
preprocessor.analysers = [ # Which basic analysers to add (flexible)
    cmm.preprocessor.TumourCount(),
    cmm.preprocessor.HypoxicCount(),
    cmm.preprocessor.NecroticCount(),
    cmm.preprocessor.MeanTumourOxygen(),
    cmm.preprocessor.MedianTumourOxygen(),
    cmm.preprocessor.MeanTumourRadius(),
    cmm.preprocessor.MedianTumourRadius(),
    cmm.preprocessor.TCellCount(),
]

import multiprocessing
import tqdm

def process_sim(sim_id):
    sim = experiment.read_simulation(sim_id)
    preprocessor.process(sim, start=0, stop=None, step=1, disable_tqdm=True)

with multiprocessing.Pool(processes=50) as pool:
    _=list(tqdm.tqdm(pool.imap(process_sim,
                               experiment.sim_ids),
                    total=len(experiment.sim_ids)))

100%|██████████| 1/1 [00:03<00:00,  3.07s/it]


The output of the preprocessor is the file `analysis_output/TCellABM/info/sim_1520.csv`

In [9]:
import pandas as pd
pd.read_csv("analysis_output/TCellABM/info/sim_1520.csv", index_col='timestep')

,n_tumour,n_tumour_hypoxic,n_tumour_necrotic,mean_tumour_oxygen,median_tumour_oxygen,mean_tumour_radius,median_tumour_radius,n_tcells
timestep,,,,,,,,
0,4.0,0.0,0.0,0.999735,0.999728,0.392259,0.392259,0.0
600,5.0,0.0,0.0,0.970322,0.969931,0.484881,0.487153,0.0
1200,5.0,0.0,0.0,0.955396,0.955028,0.484881,0.487153,0.0
1800,6.0,0.0,0.0,0.942456,0.942593,0.479744,0.487207,0.0
2400,7.0,0.0,0.0,0.927253,0.928161,0.480557,0.487193,0.0
...,...,...,...,...,...,...,...,...
57600,439.0,11.0,0.0,0.671453,0.698204,0.442896,0.455686,3147.0
58200,278.0,7.0,0.0,0.688071,0.721707,0.451667,0.461253,3229.0
58800,132.0,1.0,0.0,0.727721,0.747633,0.464546,0.468037,3308.0


# How to use Analysers

NOTE: Here we will demonstrate running analysis 'manually' for a single simulation / timepoint.  
Generally this will be in an analysis script and analysis results will be written to a database (See other example).  

Most analysis is done by `TimepointAnalysers`.  

Currently there is only one `SimulationAnalyser` (`MultiTimepointAnalyser`) which applies a given `TimepointAnalyser` to a specified set of timepoints (this allows `TimepointAnalyser` to only worry about analysing a single timepoint, other things e.g. storing the analysis results are handled by `MultiTimepointAnalyser`).  
In the future there may be more `SimulationAnalysers`.  

## Timepoint Analysers


### Tumour Roundness
Draws alpha shape around 'Tumour' cells, and calculates the circularity of this shape (see source code for details).  


In [3]:
analyser = cmm.analysers.RoundnessAnalyser(alpha=.5)
print("Analysis Name:", str(analyser)) # This is the analysis name that will be stored if using csdc database connector

result = analyser.analyse(sample_timepoint, simulation) # Returns dictionary
display(result)

Analysis Name: Roundness alpha=0.50


{'roundness': 0.9544642838472623}

### Tumour Subregions

Area of each tumour region.  
Regions defined using alpha shapes, e.g.:  
![Tumour Regions](_demo_plots/tumour-regions.png "Tumour Regions")

In [4]:
analyser = cmm.analysers.TumourRegionSizesAnalyser(alpha=.5)
print("Analysis Name:", str(analyser)) # This is the analysis name that will be stored if using csdc database connector

result = analyser.analyse(sample_timepoint, simulation)
display(result)

Analysis Name: Tumour Region Sizes alpha=0.50


{'tumour_area': 651.6778854404512,
 'normoxic_area': 305.5887651138977,
 'hypoxic_area': 346.08912032655337,
 'necrotic_area': 0.0}

### Number/Density of T-Cells in Different Tumour Regions
Regions defined using alpha shape, e.g.:  
![T-Cell Zones](_demo_plots/Tumour%20T-Cell%20Zones.png "T-Cell Zones")

In [5]:
analyser = cmm.analysers.TumourRegionTCellCountAnalyser(alpha=.5)
print("Analysis Name:", str(analyser)) # This is the analysis name that will be stored if using csdc database connector

result = analyser.analyse(sample_timepoint, simulation)
display(result)

Analysis Name: Tumour Region TCell Counts alpha=0.50 buffer_size=5


{'necrotic_region_count': 0,
 'necrotic_region_density': 0,
 'hypoxic_region_count': 35,
 'hypoxic_region_density': 0.10113002098123064,
 'normoxic_region_count': 4,
 'normoxic_region_density': 0.013089486449245402,
 'buffer_region_count': 5,
 'buffer_region_density': 0.009261920485540287,
 'exterior_region_count': 181,
 'exterior_region_density': 0.020548387013649142}

###  PCF Analysis (Requires MuSpAn)
e.g. TCell-Tumour PCF

In [6]:
analyser = cmm.analysers.TumourTCellPCFAnalyser(r_max=50, dr=1, step=1)
print("Analysis Name:", str(analyser)) # This is the analysis name that will be stored if using csdc database connector

result = analyser.analyse(sample_timepoint, simulation)
display(result.T)

Analysis Name: Tumour-TCell PCF r=50 dr=1 step=1


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
r,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,41.000000,42.000000,43.000000,44.00000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000
g,3.218791,3.339928,3.414774,3.374539,3.337044,3.149176,3.000644,2.847592,2.672528,2.481721,...,1.082237,1.054717,1.028921,1.00925,0.981697,0.962843,0.917943,0.875742,0.831998,0.781473


## MultiTimepointAnalyser (Analysis result over time)

### Roundness
Calculate roundness at every 20th simulation timepoint

In [7]:
analyser = cmm.analysers.MultiTimepointAnalyser(
    cmm.analysers.RoundnessAnalyser(alpha=.5),
    timepoint_slice=slice(None,None,-20))
print(f"Analysis name: {str(analyser)}")

result = analyser.analyse(simulation)
display(result)

Analysis name: Roundness alpha=0.50


,roundness
timestep,
60000,0.647359
48000,0.937850
36000,0.959303
24000,0.951148
12000,0.952640
0,0.785398


### Tumour Region Sizes
Calculate every 10th simulation timepoint

In [8]:
analyser = cmm.analysers.MultiTimepointAnalyser(
    cmm.analysers.TumourRegionSizesAnalyser(alpha=.5),
    timepoint_slice=slice(None,None,-10))
print(f"Analysis name: {str(analyser)}")

result = analyser.analyse(simulation)
display(result)

Analysis name: Tumour Region Sizes alpha=0.50


,tumour_area,normoxic_area,hypoxic_area,necrotic_area
timestep,,,,
60000,12.695242,12.695242,0.000000,0.0
54000,2596.251606,2424.140838,172.110769,0.0
48000,2398.991775,1375.727055,1023.264720,0.0
42000,1635.071860,664.348434,970.723426,0.0
36000,1056.535088,391.073343,665.461745,0.0
30000,651.677885,305.588765,346.089120,0.0
24000,330.849528,242.202340,88.647189,0.0
18000,110.553214,110.553214,0.000000,0.0
12000,33.084622,33.084622,0.000000,0.0
